In [1]:
import boto3
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.utils import AnalysisException
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext

session = boto3.session.Session(region_name='eu-west-1')
s3 = session.client('s3')
glue = session.client('glue')
spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').config('spark.sql.hive.convertMetastoreParquet','false').getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(SparkContext.getOrCreate())

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Trying to create a Glue session for the kernel.
Session Type: etl
Session ID: c87d800c-6c31-404d-bf86-1d665b4f545a
Applying the following default arguments:
--glue_kernel_version 1.0.6
--enable-glue-datacatalog true
Waiting for session c87d800c-6c31-404d-bf86-1d665b4f545a to get into ready status...
Session c87d800c-6c31-404d-bf86-1d665b4f545a has been created.



You are already connected to a glueetl session c87d800c-6c31-404d-bf86-1d665b4f545a.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0
